In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
train_df = pd.read_csv('./data/train.csv')
test_df = pd.read_csv('./data/test.csv')
sample_submit = pd.read_csv('./data/sample_submit.csv',header=None, index_col=0)

In [3]:
train_df = train_df.drop(columns=['id'])
df = train_df.dropna()

In [4]:
df.shape

(1600, 7)

In [5]:
df['charges'].value_counts()

0    1256
1     198
2     146
Name: charges, dtype: int64

In [6]:
df.columns

Index(['age', 'sex', 'bmi', 'children', 'smoker', 'region', 'charges'], dtype='object')

In [7]:
X = df.drop(columns=['charges'])
y = df['charges']

In [8]:
X = pd.get_dummies(X)

In [9]:
X.shape, y.shape

((1600, 11), (1600,))

In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0, stratify=y)

In [11]:
y_train.value_counts()

0    1005
1     158
2     117
Name: charges, dtype: int64

# ロジスティック回帰
何も考えずにすべての変数を突っ込んだ

In [12]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline


pipe = make_pipeline(StandardScaler(),LogisticRegression(C=1, max_iter=500))

pipe.fit(X_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('logisticregression', LogisticRegression(C=1, max_iter=500))])

In [13]:
from sklearn.metrics import accuracy_score

# 正答率
acc_train = accuracy_score(y_train, pipe.predict(X_train))
acc_test = accuracy_score(y_test, pipe.predict(X_test))

print("acc_train: "+ str(acc_train) + "   acc_test: "+ str(acc_test))

acc_train: 0.878125   acc_test: 0.859375


In [14]:
from sklearn.metrics import f1_score

f1_train = f1_score(y_train, pipe.predict(X_train), average='macro')
f1_test = f1_score(y_test, pipe.predict(X_test), average='macro')

# 評価結果の表示
print("f1_train:", f1_train, "f1_test", f1_test)

f1_train: 0.7437793556820361 f1_test 0.672745732597246


In [15]:
from sklearn.metrics import confusion_matrix

cm_train = confusion_matrix(y_train, pipe.predict(X_train))
cm_test = confusion_matrix(y_test, pipe.predict(X_test))

# 混同行列の表示
print(cm_train)
print(cm_test)

[[960  39   6]
 [ 66  75  17]
 [  7  21  89]]
[[242   7   2]
 [ 19  14   7]
 [  3   7  19]]


## 提出

In [16]:
test = test_df.drop(columns=['id'])
#print(test.shape)

#test = test.fillna(0)
#print(test.shape)

test_X = pd.get_dummies(test)
test_X.shape

(400, 11)

In [17]:
sample_submit[1] = pipe.predict(test_X)
sample_submit.to_csv('submit_lr.csv', header=None)
sample_submit.head()

,1
0,
13,0
23,2
27,0
28,0
29,0


- ダウンサンプリングしてみる<br>
- 少ない方は同じのを多い方は何度もランダムに抽出
- アンサンブル

## Featuretools を使った特徴量エンジニアリング

In [18]:
#テストデータ
import pandas as pd
data = {'name': ['a', 'b', 'c'],
        'x': [1, 2, 3],
        'y': [2, 4, 6],
        'z': [3, 6, 9],}
df_ = pd.DataFrame(data)
df_

,name,x,y,z
0,a,1,2,3
1,b,2,4,6
2,c,3,6,9


### EntitySetをつくる
まず空の`featuretools.EntitySet`を作ります。<br>
`EntitySet`はentity同士の関係や処理する内容を定義しておく為のオブジェクトですが以下ではまだidしか書かれていません。<br>
idについては省略可能ですが、以下では`id='example'`としています。

In [19]:
import featuretools as ft
es = ft.EntitySet(id='example')
es

Entityset: example
  Entities:
  Relationships:
    No relationships

### EntitySetにentityを追加
以下では先程作ったdfを登録して'locations'という名前で呼べるようにしています。<br>
`index=`はそのままindexを指定する為の引数で、省略するとDataFrameの最初の列をindexとして扱うようになっています。

In [20]:
es = es.entity_from_dataframe(entity_id='locations',
                              dataframe=df_,
                              index='name')
es

Entityset: example
  Entities:
    locations [Rows: 3, Columns: 4]
  Relationships:
    No relationships

EntitySetに登録されているentityは以下のように呼び出すことができます

In [21]:
es['locations']

Entity: locations
  Variables:
    name (dtype: index)
    x (dtype: numeric)
    y (dtype: numeric)
    z (dtype: numeric)
  Shape:
    (Rows: 3, Columns: 4)

In [22]:
es['locations'].df

,name,x,y,z
a,a,1,2,3
b,b,2,4,6
c,c,3,6,9


### dfsを実行する
`EntitySet`ができたので、後は`ft.dfs()`に渡すだけで特徴量を作ってくれます。<br>
`target_entity=`はメインとするentity、`trans_primitives=`が特徴量間の組み合わせに適用する計算方法、`agg_primitives=`がaggregateに使う計算方法です。

使用可能なprimitiveは以下にまとまっています<br>
https://primitives.featurelabs.com/

以下では`add_numeric`で特徴量間の和を、`subtract_numeric`で特徴量間の差を追加するように指示しています



In [23]:
feature_matrix, feature_defs = ft.dfs(entityset=es,
                                      target_entity='locations',
                                      trans_primitives=['add_numeric', 'subtract_numeric'],
                                      agg_primitives=[],
                                      max_depth=1)
feature_matrix

,x,y,z,x + y,x + z,y + z,x - y,x - z,y - z
name,,,,,,,,,
a,1,2,3,3,4,5,-1,-2,-1
b,2,4,6,6,8,10,-2,-4,-2
c,3,6,9,9,12,15,-3,-6,-3


### 3-2. aggregateのみ
agg_primitivesにはaggregateをつくる計算方法を指定します

In [24]:
#テストデータ
data = {'item_id': [1, 2, 3, 4, 5],
        'name': ['apple', 'broccoli', 'cabbage', 'dorian', 'eggplant'],
        'category': ['fruit', 'vegetable', 'vegetable', 'fruit', 'vegetable'],
        'price': [100, 200, 300, 4000, 500]}
item_df = pd.DataFrame(data)
item_df

,item_id,name,category,price
0,1,apple,fruit,100
1,2,broccoli,vegetable,200
2,3,cabbage,vegetable,300
3,4,dorian,fruit,4000
4,5,eggplant,vegetable,500


In [25]:
#EntitySetをつくる
es = ft.EntitySet(id='example')
es = es.entity_from_dataframe(entity_id='items',
                              dataframe=item_df,
                              index='item_id')
es

Entityset: example
  Entities:
    items [Rows: 5, Columns: 4]
  Relationships:
    No relationships

ここにaggregateに使う為のrelationshipを追加します。<br>
以下ではitemsというentityをベースに、新たにcategoryというentityを作り、そのときのindexをcategoryとせよという指示になっています。

In [26]:
es = es.normalize_entity(base_entity_id='items',
                         new_entity_id='category',
                         index='category')
es

Entityset: example
  Entities:
    items [Rows: 5, Columns: 4]
    category [Rows: 2, Columns: 1]
  Relationships:
    items.category -> category.category

これでどうなっているかというと、まずitemsは入れたそのままになっています。

In [27]:
es['items'].df

,item_id,name,category,price
1,1,apple,fruit,100
2,2,broccoli,vegetable,200
3,3,cabbage,vegetable,300
4,4,dorian,fruit,4000
5,5,eggplant,vegetable,500


In [28]:
es['category'].df

,category
fruit,fruit
vegetable,vegetable


dfsを実行する<br>
agg_primitivesにcount, sum, meanを指定して実行してみます

In [29]:
feature_matrix, feature_defs = ft.dfs(entityset=es,
                                      target_entity='items',
                                      trans_primitives=[],
                                      agg_primitives=['count', 'sum', 'mean'],
                                      max_depth=2)
feature_matrix

,name,category,price,category.COUNT(items),category.MEAN(items.price),category.SUM(items.price)
item_id,,,,,,
1,apple,fruit,100,2,2050.000000,4100
2,broccoli,vegetable,200,3,333.333333,1000
3,cabbage,vegetable,300,3,333.333333,1000
4,dorian,fruit,4000,2,2050.000000,4100
5,eggplant,vegetable,500,3,333.333333,1000


aggregateできる列がitems.priceしかないのでCOUNT(), category.MEAN(), category.SUM()がそれぞれ算出されて、3列追加されたDataFrameができました。

## ダウンサンプリング

In [30]:
# 学習用データの説明変数と目的変数を結合
train = pd.concat([X_train,y_train],axis=1)

# 完済のデータと貸し倒れのデータを別々の変数に代入
fp = train[train['ChargedOff'] == 0]
co = train[train['ChargedOff'] == 1]

# 貸し倒れのデータ数と同じ数だけ完済のデータをランダムに取り出し
fp = fp.sample(n=co.shape[0], random_state=0)

# 完済のデータと貸し倒れのデータを縦結合
train = pd.concat([fp, co])

# 説明変数と目的変数をそれぞれ別の変数に代入
X_train = train.drop(columns=['ChargedOff'])
print(X_train.shape)

y_train = train['ChargedOff']
print(y_train.shape)

KeyError: 'ChargedOff'